In [ ]:
import pandas as pd
from classes.WindAPI import *
from sqlalchemy import *
from sqlalchemy.orm import Session
from create_ref_mariadb import mariadb_connection, windturbines
from create_ref_mongodb import mongodb_connection
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import numpy as np
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import pytz

In [ ]:
def fetch_windturbines_data(engine):
    # Récupérer les identifiants et last scada update de turbines depuis la table windturbine dans MariaDB
    stmt = select(windturbines.c['windturbine_id','last_scada_update']).filter(windturbines.c.last_scada_update.isnot(None))
    turbines = []
    with Session(engine) as session:
        for row in session.execute(stmt):
            turbines.append(row)

    # Traiter les données SCADA pour chaque turbine
    return turbines

def fetch_scada_data(turbineId):
    pass

client_mongodb = mongodb_connection()
engine_mariadb = mariadb_connection()
turbines = fetch_windturbines_data(engine_mariadb)


lastday_datetime = datetime.now() - timedelta(hours=24)
tz = pytz.timezone('Europe/Paris')
current_datetime = datetime.now(tz)


scada_col = client_mongodb.scada
end = datetime(2014, 9, 24, 7, 52, 4)
for turbine in turbines:
    scada_data = scada_col.find({
        "windturbine_id": turbine.windturbine_id,
        "log_date": {'$gte': end}
    })
    df = pd.DataFrame(list(scada_data))
    print(df.head())